In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt

# Research Question
***Should you work for a smaller company instead of a larger one?***
 - Company size
 - Salary Estimate
 - Rating 

"More for visulization"
 - Job Title
 - Industry and Sector 
 - Location 

In [3]:
df1 = pd.read_csv('data/raw/DataEngineer.csv',sep=',') # Comma-seperated values file and Comma-delimited 
df1.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,Data Engineer,$80K-$150K (Glassdoor est.),Company Description\nSagence is a management a...,4.5,Sagence\n4.5,"New York, NY","Chicago, IL",1 to 50 employees,2009,Company - Private,Consulting,Business Services,$10 to $25 million (USD),"WCI Consulting, PwC",-1
1,Senior Data Engineer (Healthcare Domain experi...,$80K-$150K (Glassdoor est.),"Key Responsibilities\n\n- Architect, build, an...",3.4,Enterprise Integration\n3.4,"New York, NY","Jacksonville, FL",51 to 200 employees,1998,Company - Private,IT Services,Information Technology,$25 to $50 million (USD),-1,-1
2,Data Engineers,$80K-$150K (Glassdoor est.),Overview\n\nJob description\n\nPosition Overvi...,5.0,Maestro Technologies\n5.0,"New York, NY","Trenton, NJ",51 to 200 employees,2003,Company - Private,IT Services,Information Technology,$5 to $10 million (USD),-1,-1
3,Client Trade Support Engineer,$80K-$150K (Glassdoor est.),About the Position\n\n\nThis position will sup...,4.8,Jane Street\n4.8,"New York, NY","New York, NY",501 to 1000 employees,2000,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,-1,-1
4,Data Engineer,$80K-$150K (Glassdoor est.),Data Engineer\n\nJob Details\nLevel\nExperienc...,3.7,GNY Insurance Companies\n3.7,"New York, NY","New York, NY",201 to 500 employees,1914,Company - Private,Insurance Carriers,Insurance,$100 to $500 million (USD),"Travelers, Chubb, Crum & Forster",True


In [16]:
# Removing the "Competitors" and "Easy Apply" because there are too many "-1" which I assumed is unknown or NaN

df2 = df1.drop(['Competitors','Easy Apply'],axis='columns') 
df2.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Engineer,$80K-$150K (Glassdoor est.),Company Description\nSagence is a management a...,4.5,Sagence\n4.5,"New York, NY","Chicago, IL",1 to 50 employees,2009,Company - Private,Consulting,Business Services,$10 to $25 million (USD)
1,Senior Data Engineer (Healthcare Domain experi...,$80K-$150K (Glassdoor est.),"Key Responsibilities\n\n- Architect, build, an...",3.4,Enterprise Integration\n3.4,"New York, NY","Jacksonville, FL",51 to 200 employees,1998,Company - Private,IT Services,Information Technology,$25 to $50 million (USD)
2,Data Engineers,$80K-$150K (Glassdoor est.),Overview\n\nJob description\n\nPosition Overvi...,5.0,Maestro Technologies\n5.0,"New York, NY","Trenton, NJ",51 to 200 employees,2003,Company - Private,IT Services,Information Technology,$5 to $10 million (USD)
3,Client Trade Support Engineer,$80K-$150K (Glassdoor est.),About the Position\n\n\nThis position will sup...,4.8,Jane Street\n4.8,"New York, NY","New York, NY",501 to 1000 employees,2000,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable
4,Data Engineer,$80K-$150K (Glassdoor est.),Data Engineer\n\nJob Details\nLevel\nExperienc...,3.7,GNY Insurance Companies\n3.7,"New York, NY","New York, NY",201 to 500 employees,1914,Company - Private,Insurance Carriers,Insurance,$100 to $500 million (USD)


In [13]:
unique_job = list(set(df1["Job Title"]))
unique_job[0:15]

['Software Engineer, Data',
 'Senior Software Engineer - Compute Infrastructure',
 'Staff Engineer, Device Modeling',
 'Data Center Hardware Engineer (Deployment)',
 'Analog Design Engineer',
 'Sr Test Platform Engineer',
 'Sr. Data Engineer',
 'Lead Security Engineer',
 'Systems Engineer',
 'Senior Engineer, Data',
 'Java Software Engineer - Server Side',
 'Senior Backend Engineer - Chicago',
 'Java with big data engineer',
 'Infrastructure Engineer',
 'SR. IT Engineer']

In [ ]:
# There are a lot of different unique job title, but with similiar position (i.e Data Engineer & Data Engineer Python)
# In order to use a count plot with job title, we would have to organize similiar job title into one catergory and then show the count in that catergory and its corresponding info(i.e Rating Salary)
# Or the dataset might be large enough to only include the job with basic title (i.e Data Engineer Software Engineer)
# Or we can make the job title more general, instead of "Data Engineer - Python" it can be "Data Engineer", its corresponding information will make it unique from the others Data Engineer titles


In [18]:
unique_size = set(df1["Size"])
unique_size

{'-1',
 '1 to 50 employees',
 '10000+ employees',
 '1001 to 5000 employees',
 '201 to 500 employees',
 '5001 to 10000 employees',
 '501 to 1000 employees',
 '51 to 200 employees',
 'Unknown'}

In [ ]:
# I am going to categorize the size of the company into using this Canadian goverment website with small modifcation to work with this data set
# https://www150.statcan.gc.ca/n1/pub/11f0027m/2011069/part-partie1-eng.htm
# The exact number of the company size is not known in this data set so I would say it's okay to go over the limit that was described in the goverment link
# Also the goverment link is archived so the infornmation is not updated. This is just for use of referencing

# 1 to to 200 is small
# 201 to 500 is medium
# +500 is large
# -1 is assumed to be unknown 

# Method Chaining